In [44]:
import pandas as pd
from ipywidgets import widgets as wg
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime as dt
from plotly.offline import init_notebook_mode, iplot
import numpy as np

init_notebook_mode()


In [169]:
## Explanation for PDBD
## interval_name: Name of interval
## min_ma = Max milions of years ago
## max_ma = Max milions of years ago
## X_bt = number of genera with ranges crossing both top and bottom boundary of interval
## X_bL = number of genera with a last occurrence in the interval and first occurrence prior to interval
## X_Ft = number of genera with a first occurrence in the interval and a last occurrence after the interval
## X_FL = number of genera witha first and last occurrence in the interval
## n_occs = Total number of occurances

df = pd.read_csv('pbdb/pbdb_diversity_over_time.csv')
df.head()

interval_no       interval_name  max_ma  min_ma  X_Ft  X_bL  X_FL  X_bt   
0         3002            Holocene  0.0117  0.0000     0  2193   579     0  \
1          922    Late Pleistocene  0.1260  0.0117   256  1671   553  1937   
2          923  Middle Pleistocene  0.7810  0.1260   194   415   183  3414   
3          740           Calabrian  1.8060  0.7810   254   553   168  3575   
4          741            Gelasian  2.5880  1.8060    95   164    52  4033   

   sampled_in_bin  implied_in_bin  n_occs  
0            2772             107   17695  
1            3848              95   38811  
2            2538              70   16300  
3            2825              83   23523  
4            1195              45    5888

In [171]:
# Drop the current era timeframes
df = df.iloc[3:]
df.head()

interval_no interval_name  max_ma  min_ma  X_Ft  X_bL  X_FL  X_bt   
3          740     Calabrian   1.806   0.781   254   553   168  3575  \
4          741      Gelasian   2.588   1.806    95   164    52  4033   
5           96    Piacenzian   3.600   2.588   151   577   195  4046   
6           97      Zanclean   5.333   3.600   398   602   296  4225   
7           98     Messinian   7.246   5.333   141   308   187  4686   

   sampled_in_bin  implied_in_bin  n_occs  
3            2825              83   23523  
4            1195              45    5888  
5            2852              97   20188  
6            3544              96   24850  
7            2150             107   12172

In [176]:
# years from current to 541 MA in millions of years
years = range(1, int(df["max_ma"].max())) 

In [191]:
# Create a new dataframe, using millions of years get last occurence in interval for species extinction events
dfExtinct = pd.DataFrame(years, columns=["years ma"])

#dfExtinct["extinct"] = 
dfExtinct.head()

years ma
0         1
1         2
2         3
3         4
4         5

In [181]:
df.query(f'max_ma >= 1')

interval_no interval_name   max_ma   min_ma  X_Ft  X_bL  X_FL  X_bt   
3           740     Calabrian    1.806    0.781   254   553   168  3575  \
4           741      Gelasian    2.588    1.806    95   164    52  4033   
5            96    Piacenzian    3.600    2.588   151   577   195  4046   
6            97      Zanclean    5.333    3.600   398   602   296  4225   
7            98     Messinian    7.246    5.333   141   308   187  4686   
..          ...           ...      ...      ...   ...   ...   ...   ...   
95         1117       Stage 5  509.000  504.500    13    33    22   203   
96         1118       Stage 4  514.000  509.000    59   142   252   177   
97         1119       Stage 3  521.000  514.000   221   152   870    98   
98         1120       Stage 2  529.000  521.000   142    34    57   108   
99         1121     Fortunian  541.000  529.000   142     0    63     0   

    sampled_in_bin  implied_in_bin  n_occs  
3             2825              83   23523  
4             1195              45    5888  
5             2852              97   20188  
6             3544              96   24850  
7             2150             107   12172  
..             ...             ...     ...  
95              99               2     428  
96             514               0    4288  
97            1308               7   13879  
98             309               1    2195  
99             205               1    1141  

[97 rows x 11 columns]

In [217]:
# Get the data for each time slice
dfExtinct["first_and_last_in_age"] = dfExtinct["years ma"].apply(lambda x: df.query(f'max_ma >= {x}').head(1).get("X_FL").values[0])
dfExtinct["last_in_age"] = dfExtinct["years ma"].apply(lambda x: df.query(f'max_ma >= {x}').head(1).get("X_bL").values[0])
dfExtinct["occurences"] = dfExtinct["years ma"].apply(lambda x: df.query(f'max_ma >= {x}').head(1).get("n_occs").values[0])
dfExtinct["number_extinct"] = dfExtinct["first_and_last_in_age"] + dfExtinct["last_in_age"]
dfExtinct.head()



years ma  first_and_last_in_age  last_in_age  number_extinct  occurences
0         1                    168          553             721       23523
1         2                     52          164             216        5888
2         3                    195          577             772       20188
3         4                    296          602             898       24850
4         5                    296          602             898       24850

In [236]:
# instantiate the figure object
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": False}]])

# add a scatter trace for every column
# Add traces
fig.add_trace(
    go.Scatter(x=dfExtinct["years ma"], y=dfExtinct["occurences"], name="Years MA")
)

# Add annotations for major extinction events
fig.add_annotation(
        x=65,
        y=dfExtinct.iloc[65]["occurences"],
        xref="x",
        yref="y",
        text="End Cretaceous 65ma",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=0,
        ay=60,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )

# Add annotations for major extinction events
fig.add_annotation(
        x=200,
        y=dfExtinct.iloc[200]["occurences"],
        xref="x",
        yref="y",
        text="End Triassic 200ma",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=0,
        ay=-200,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )

fig.add_annotation(
        x=250,
        y=dfExtinct.iloc[250]["occurences"],
        xref="x",
        yref="y",
        text="End Permian 250ma",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=0,
        ay=60,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )


fig.add_annotation(
        x=360,
        y=dfExtinct.iloc[360]["occurences"],
        xref="x",
        yref="y",
        text="Late Devonian 360ma",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=0,
        ay=-60,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )


fig.add_annotation(
        x=444,
        y=dfExtinct.iloc[444]["occurences"],
        xref="x",
        yref="y",
        text="End Ordovician 444ma",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=0,
        ay=-220,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )

# Set x-axis title
fig.update_xaxes(title_text="<b>Millions of Years Ago</b>")

# Set y-axes titles
fig.update_yaxes(title_text="<b>No. species recorded</b>", secondary_y=False)

# change the scale to logarithmic and add title
fig.update_layout(
     template='simple_white',
     height=500,
     margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()